# Data Processing and Feature Engineering

In this notebook, we merge the different datasets together, select features and engineer some new features. This first step in feature selection and engineering is based on my knowledge of the data. 

We proceed in this order:  

1. Filter water systems of interest (active water systems in New England)
2. Select features of interest for water systems, and data cleaning
3. Filter violations of interest (pesticides)
4. Select features of interest for violations, and data cleaning
5. Add estimated pesticide use for the water systems
6. Merge water systems and violations to obtain violations by water systems (!)
7. 
8. Engineer new features of interest for violations by water systems 


In the end, we obtain a dateset on which we can do model training and selection. In a later step, when trying different models, a new loop of feature selection and engineering might be needed, and will be performed in a separate notebook.     


In [194]:
import pandas as pd
import numpy as np
import datetime
import time

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import geopandas as gpd # to identify neighboring water systems via ZIP code
from shapely.geometry import Point, Polygon

## Water Systems

We did already select the water systems of interest: EPA region 01, New England, and only active water systems.

In [195]:
# load water systems:
ws_raw = pd.read_csv('../data/active_water_systems_NewEngland.csv')
ws_raw.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
pwsid,ME0004628,ME0092288,ME0009198,ME0094505,ME0007311
pws_name,MACHIAS TRAILER PARK,MARSH BROOK ESTATES,TRAILS END STEAK HOUSE & TAVERN,R & R VACATION HOME PARK,NOKOMIS CAMPING AREA LLC
npm_candidate,Y,Y,Y,Y,Y
primacy_agency_code,ME,ME,ME,ME,ME
epa_region,1,1,1,1,1
season_begin_date,NaN,NaN,01-01,01-01,05-01
season_end_date,NaN,NaN,12-31,12-31,09-30
pws_activity_code,A,A,A,A,A
pws_deactivation_date,NaN,NaN,NaN,NaN,NaN


In [215]:
# DATA CLEANING FOR WATER SYSTEMS
ws = ws_raw.copy()

# IMPORTANT NOTE: wer cannot use the ZIP code to locate the water systems, 
# as it is sometimes the ZIP code of the legal entity, which is not necessarily in the same place.
# ==> we can locate it with the ansi_entity_code.

# We remove the 6 water systems without localization (no ansi_entity_code)
# (It turns out they are tribal lands) ==> ! introduction of a potential bias
# ws_raw.ansi_entity_code.isnull().sum() 
ws = ws[~ws['ansi_entity_code'].isnull()]

# verifying that the counties_served from the water system table is the same as the county_served from geography:
# (ws.county_served != ws.counties_served).sum()
# so deleting one:
ws.drop('county_served', axis=1, inplace=True)

ws.shape

(10476, 49)

In [216]:
# First Filter to select needed columns

select_features_ws = ['pwsid', 'pws_name', 'primacy_agency_code', 'pws_type_code', 
                      'gw_sw_code', # if the water system is considered having ground water (“gw”) or surface water (‘sw”) source under SDWA.
                     'owner_type_code', 'population_served_count', 'primary_source_code',
                      'is_wholesaler_ind', # whether the system is a wholesaler of water.
                     'is_school_or_daycare_ind', # if the water system’s primary service area is a school or daycare
                     'service_connections_count', 
                      'source_water_protection_code', # N: WS has not implemented source water protection according to state policy. Y: WS has substantially implemented.
                     # ! source_water_protection_code: most Y only after 01-JAN-2012
                      'cities_served', 'counties_served', 'ansi_entity_code']

ws = ws.loc[:, select_features_ws] # keep only selected columns
ws.head()


,pwsid,pws_name,primacy_agency_code,pws_type_code,gw_sw_code,owner_type_code,population_served_count,primary_source_code,is_wholesaler_ind,is_school_or_daycare_ind,service_connections_count,source_water_protection_code,cities_served,counties_served,ansi_entity_code
0,ME0004628,MACHIAS TRAILER PARK,ME,CWS,GW,P,65,GW,N,N,26,N,MACHIAS,Washington,29.0
1,ME0092288,MARSH BROOK ESTATES,ME,CWS,GW,P,70,GW,N,N,28,N,SANFORD,York,31.0
2,ME0009198,TRAILS END STEAK HOUSE & TAVERN,ME,TNCWS,GW,P,390,GW,N,N,1,N,EUSTIS,Franklin,7.0
3,ME0094505,R & R VACATION HOME PARK,ME,TNCWS,GW,P,40,GW,N,N,1,N,NAPLES,Cumberland,5.0
4,ME0007311,NOKOMIS CAMPING AREA LLC,ME,TNCWS,GW,P,118,GW,N,N,1,N,HARRISON,Cumberland,5.0


In [344]:
ws['ansi_entity_code'] = ws['ansi_entity_code'].astype(object)
ws.dtypes

pwsid                           object
pws_name                        object
primacy_agency_code             object
pws_type_code                   object
gw_sw_code                      object
owner_type_code                 object
population_served_count          int64
primary_source_code             object
is_wholesaler_ind               object
is_school_or_daycare_ind        object
service_connections_count        int64
source_water_protection_code    object
cities_served                   object
counties_served                 object
ansi_entity_code                object
dtype: object

### Adding Geographic Information to the Water Systems

TO BE DONE?
* = add the shape from shapefile
* = add Lat/Lon from county (centroid)



## Violations

We already filter the data by year, because we are only interested in recent violations. The number of observed violations greatly increased to reach a new plateau in 2009 (known from [previous work](https://github.com/de-la-viz/US-Public-Water-Systems/blob/master/US%20Drinking%20Water%20Quality%20Violations.ipynb)) because of the introduction of new guidelines and rules. We will thus focus on violations from 2009 onwards. 

In [371]:
# load the violations:
violations_raw = pd.read_csv('../data/violations_NewEngland.csv', dtype='object')
# violations_raw.head().T

In [372]:
# SOME FILTERING AND DATA CLEANING:

violations = violations_raw.copy()

# transform the dates to datetime:
violations.rtc_date = pd.to_datetime(violations.rtc_date)
violations.compl_per_begin_date = pd.to_datetime(violations.compl_per_begin_date)
violations.compl_per_end_date = pd.to_datetime(violations.compl_per_end_date)

violations.loc[:,'year'] = violations['compl_per_begin_date'].dt.year # year when the violation was discovered
violations.loc[:,'month'] = violations['compl_per_begin_date'].dt.month # month when the violation was discovered

# create new column with quarters:
def by_quarter(row):
    if row['month'] < 4:
        return 1
    elif row['month'] >= 4 and row['month'] < 7:
        return 2
    elif row['month'] >= 7 and row['month'] < 10:
        return 3
    else:
        return 4
violations.loc[:,'quarter'] = violations.apply(by_quarter, axis=1) 


select_features_viol = ['pwsid', 'violation_id', 'violation_code', 'violation_category_code', 
                       'is_health_based_ind', 'contaminant_code', 'is_major_viol_ind',
                       'rule_group_code',
                       'year', 'month', 'quarter']

violations = violations.loc[:, select_features_viol] # keep only selected columns

# Note: 91 contaminant_code are empty.
# violations.contaminant_code.isnull().sum()

print(violations.shape)
violations.head().T


(65534, 11)


,0,1,2,3,4
pwsid,ME0094672,ME0009683,ME0000625,ME0000625,ME0000625
violation_id,157508,60007,6318,6316,6315
violation_code,22,22,75,27,27
violation_category_code,MCL,MCL,Other,MR,MR
is_health_based_ind,Y,Y,N,N,N
contaminant_code,3100,3100,7500,2456,2950
is_major_viol_ind,NaN,NaN,NaN,Y,Y
rule_group_code,100,100,400,200,200
year,2008,2014,2014,2011,2011
month,6,8,10,1,1


In [373]:
violations.contaminant_code.isnull().sum() # 91 violations without specified contaminant. those will be lost...

91

### Adding the Contaminant Names

In [374]:
# loading the contaminants codes:
contaminants = pd.read_csv('../data/contaminant_codes.csv')
# contaminants.dtypes
# contaminants.head()

In [375]:
# merging the name to the violations
violations_cont = violations.merge(contaminants, how='left', on='contaminant_code')

The most common violations:

In [376]:
# most occuring violations in New Hampshire:
# (not all are contaminants, nor pesticides)
violations_cont.contaminant_name.value_counts().head(50)

COLIFORM (TCR)                   13108
PUBLIC NOTICE                     4533
LEAD & COPPER RULE                3440
CONSUMER CONFIDENCE RULE          2409
NITRATE                           2162
E. COLI                            901
ARSENIC                            838
TTHM                               665
NITRITE                            662
TOTAL HALOACETIC ACIDS (HAA5)      619
TETRACHLOROETHYLENE                572
VINYL CHLORIDE                     564
TRANS-1,2-DICHLOROETHYLENE         564
1,2-DICHLOROETHANE                 564
TOLUENE                            561
DICHLOROMETHANE                    561
P-DICHLOROBENZENE                  561
1,1-DICHLOROETHYLENE               560
CIS-1,2-DICHLOROETHYLENE           560
1,2-DICHLOROPROPANE                560
CARBON TETRACHLORIDE               559
TRICHLOROETHYLENE                  559
STYRENE                            559
BENZENE                            559
1,1,1-TRICHLOROETHANE              558
O-DICHLOROBENZENE        

### Selection of the Contaminants of Interest: Pesticides



In [377]:
# some contaminants name are empty:
violations_cont['contaminant_name'].isnull().sum()

2721

In [378]:
# let's check those and find why:
empty_cont_name = violations_cont[violations_cont['contaminant_name'].isnull() == True]
# empty_cont_name.contaminant_code.isnull().sum() # only 91 miss a contaminant code
empty_cont_name.contaminant_code.value_counts()
# 8000: not found ==> it is the "Revised Total Coliform Rule", 
# c.f: https://www.epa.gov/sites/production/files/2018-06/documents/2017_annual_dc_drinking_water_compliance_report_508_0.pdf 


8000    2630
Name: contaminant_code, dtype: int64

We add a new column to the violations to identify pesticides

In [379]:
# list of pesticides:
pesticide_use_2009_14 = pd.read_csv('../data/pesticide_use/pesticide_use_2009_14.csv')
pesticide_use_2015 = pd.read_csv('../data/pesticide_use/2015PreliminaryEstimates/EPest.county.estimates.2015.txt', sep='\t')
pesticide_use_2016 = pd.read_csv('../data/pesticide_use/2106PreliminaryEstimates/EPest.county.estimates.2016.txt', sep='\t')
pesticide_use_2017 = pd.read_csv('../data/pesticide_use/2017PreliminaryEstimatesNoCA/EPest.county.estimates_noCA.2017.txt', sep='\t')

# append to previous years:
pesticide_use_2009_17 = pesticide_use_2009_14.append(pesticide_use_2015, ignore_index=True)
pesticide_use_2009_17 = pesticide_use_2009_17.append(pesticide_use_2016, ignore_index=True)
pesticide_use_2009_17 = pesticide_use_2009_17.append(pesticide_use_2017, ignore_index=True)


In [380]:
pesticides = pesticide_use_2009_17.COMPOUND.unique() # a list of all pesticides

In [381]:
def is_pesticide(row):
    if row['contaminant_name'] is None:
        return 0
    elif row['contaminant_name'] in pesticides:
        return 1
    else:
        return 0
violations_cont.loc[:,'is_pesticide'] = violations_cont.apply(is_pesticide, axis=1)


In [382]:
violations_cont.is_pesticide.value_counts()

0    62008
1     3526
Name: is_pesticide, dtype: int64

## Merging all SDWIS Together

We want to create a dataframe with all the water systems, and, for all the years of interest, a binary variable that say if they had seen a violation or not.  

We start by creating "repeating" the water systems data for all years:


In [383]:
years_of_interest = [2013, 2014, 2015, 2016, 2017, 2018]
ws_years = ws.copy() # water systems data
ws_years.loc[:, 'year'] = 2012 # add column with initial year
for year_ in years_of_interest:
    ws_thisyear = ws.copy() # water systems data
    ws_thisyear.loc[:, 'year'] = year_
    ws_years = ws_years.append(ws_thisyear, ignore_index=True)
print(ws_years.shape)
# ws_years.head()
# ws_years.year.value_counts()

(73332, 16)


We have now a dataset with all the information about the water systems repeated for all the years of interest. We need to add new columns to this data framt that indicate if there was a violation in a given year. In a second step we can engineer new features, for instance indicating past violations.


In [451]:
# in this chunk, we the violations and pesticide violations by water system and year

# group by year and water system to count the number of violations:
yearly_viol_count = violations_cont.groupby(['pwsid', 
                         'year'], as_index=False).count()[['pwsid', 
                                                           'year', 
                                                           'violation_id']]
yearly_viol_count = yearly_viol_count.rename(index=str, columns={"violation_id": "n_viol"})

# group by year and water system to count the number of pesticide violations:
# we first have to keep only pesticides violations:
pesticide_violations = violations_cont[violations_cont.is_pesticide == 1]
yearly_viol_pesticide_count = pesticide_violations.groupby(['pwsid', 
                         'year'], as_index=False).count()[['pwsid', 
                                                           'year', 
                                                           'violation_id']]
yearly_viol_pesticide_count = yearly_viol_pesticide_count.rename(index=str, 
                                                                 columns={"violation_id": "n_pesticide_viol"})


In [456]:
# Then we merge the count of violations to the water systems, to add this information in a new column
ws_years_viol = ws_years.merge(yearly_viol_count, how='left', on=(['pwsid', 'year']))
ws_years_viol = ws_years_viol.merge(yearly_viol_pesticide_count, how='left', on=(['pwsid', 'year']))
ws_years_viol.head().T

,0,1,2,3,4
pwsid,ME0004628,ME0092288,ME0009198,ME0094505,ME0007311
pws_name,MACHIAS TRAILER PARK,MARSH BROOK ESTATES,TRAILS END STEAK HOUSE & TAVERN,R & R VACATION HOME PARK,NOKOMIS CAMPING AREA LLC
primacy_agency_code,ME,ME,ME,ME,ME
pws_type_code,CWS,CWS,TNCWS,TNCWS,TNCWS
gw_sw_code,GW,GW,GW,GW,GW
owner_type_code,P,P,P,P,P
population_served_count,65,70,390,40,118
primary_source_code,GW,GW,GW,GW,GW
is_wholesaler_ind,N,N,N,N,N
is_school_or_daycare_ind,N,N,N,N,N


## Creating Outcome Variables

We can use the count of number of violations per year and water system - *n_viol* - or the same but only for violations due to the presence of pesticides in the water - *n_pesticide_viol* as continuous outcome variables.  

We will create two more outcomes variables that we will use for classification: a binarization of the two previous ones. *had_violation* if a water system saw a (one or more) drinking water violation in the given year, and *had_pesticide_violation* if there was a (one or more) violation due to the presence of pesticides.  

The number of water systems that saw a violation due to the presence of pesticide in the drinking water above the MCL is probably to low for training a good model, but we will see...

In [463]:
ws_years_viol.loc[:,'had_violation'] = np.where(ws_years_viol.n_viol >= 0, 1, 0)
ws_years_viol.loc[:,'had_pesticide_violation'] = np.where(ws_years_viol.n_pesticide_viol >= 0, 1, 0)

# TO DO :

* replace the NaN in n_viol and n_pesticide_viol by zeros
* add a variable if saw a violation previous year.
* save the dataframe ws_years_viol and load it in another notebook